In [131]:
from gensim.models import KeyedVectors
from gensim import matutils
import re
import os
from tqdm import tqdm_notebook as tqdm

from gensim.models.doc2vec import Doc2Vec

import numpy as np
import json

import operator

In [16]:
def preprocessing(par):
    par = par.replace('.', ' ')
    par = par.replace(',', ' ')
    par = par.replace(':', ' ')
    par = par.replace(';', ' ')
    par = par.replace('-', ' ')
    par = par.replace('+', ' ')
    par = par.replace('м²', ' ')
    par = par.replace(')', ' ')
    par = par.replace('(', ' ')
    par = par.replace('\\', ' ')
    par = par.replace('/', ' ')
    par = par.replace('"', ' ')
    par = par.replace('!', ' ')
    par = par.replace('?', ' ')
    par = par.replace('\n', ' ')
    par = par.replace('\xa0', ' ')
    par = par.replace('\u200b', ' ')
    par = par.replace('\r', ' ')
    par = re.sub('[0-9]','', par)
    par = par.replace('   ', ' ')
    par = par.replace('  ', ' ')
    par = par.strip(' ')
    par = par.lower()
        
    return par

In [147]:
def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    sim = np.dot(v1_norm, v2_norm)
    if sim is not None:
        return sim
    else:
        return 0

In [79]:
with open('Train_links.txt', 'r') as f:
    my_links = f.readlines()

### W2V

In [27]:
model_path = r"../all.norm-sz100-w10-cb0-it1-min100.w2v"
model_w = KeyedVectors.load_word2vec_format(model_path, binary=True, unicode_errors='ignore')

In [28]:
def par_t_v(par):
    vecs = np.zeros(100)
    par = list(filter(None, par.split(' ')))
    for word in par:
        try:
            vec = model_w.get_vector(word)
        except Exception as ex:
            continue
        if vecs.size is 0:
            vecs = vec
        else:
            vecs = vecs+vec
    vec = vecs/len(par)
    return vec

In [288]:
def get_w2v_vectors(text, notice_vecs_w, link):
    par = preprocessing(text)
    par = re.sub('размещено .*? текст объявления', '', par)
    vec_par = par_t_v(par)
    notice_vecs_w[link] = vec_par.tolist()
    return notice_vecs_w

In [50]:
def write_json(notice_vecs_w, global_num):
    p = r'json_w\all_json_w2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(notice_vecs_w, f)
    return

In [289]:
notice_vecs_w = {}
global_num = 1

for idx in tqdm(range(0, len(my_links))):
    path_file = 'All_files\\'+my_links[idx].strip('\n, /')+'.txt'
    if os.path.isfile(path_file):
        with open (path_file, 'r', encoding='UTF-8') as f:
            text = f.read()
        notice_vecs_w = get_w2v_vectors(text, notice_vecs_w, my_links[idx].strip('\n,/'))
        
    if (idx+1)%300 is 0 or idx is len(my_links)-1:
        write_json(notice_vecs_w, global_num)
        notice_vecs_w = {} 
        global_num+=1

A Jupyter Widget

### D2V

In [231]:
model_d = Doc2Vec.load('Doc2Vec_100s_1000e')

In [232]:
def doc_t_v(par):
    vec = np.zeros(100)
    try:
        vec = model_d.infer_vector(par)
    except Exception as ex:
        print(ex)
    return vec

In [294]:
def get_d2v_vectors(text, notice_vecs_d, link):
    par = preprocessing(text)
    par = re.sub('размещено .*? текст объявления', '', par)
    vec_par = doc_t_v(par)
    notice_vecs_d[link] = vec_par.tolist()
    return notice_vecs_d

In [235]:
def write_json(notice_vecs_d, global_num):
    p = r'json_d\all_json_d2v_' + str(global_num) +'.txt'
    with open(p, 'w') as f:
        json.dump(notice_vecs_w, f)
    return

In [295]:
notice_vecs_d = {}
global_num = 1

for idx in tqdm(range(0, len(my_links))):
    path_file = 'All_files\\'+my_links[idx].strip('\n, /')+'.txt'
    if os.path.isfile(path_file):
        with open (path_file, 'r', encoding='UTF-8') as f:
            text = f.read()
        notice_vecs_w = get_d2v_vectors(text, notice_vecs_d, my_links[idx].strip('\n,/'))
        
    if (idx+1)%300 is 0 or idx is len(my_links)-1:
        write_json(notice_vecs_w, global_num)
        notice_vecs_d = {} 
        global_num+=1

A Jupyter Widget